План проекта
1. Отобрать 9 популяций:
- Africa (393)
    - yoruba
    - bantu
    - egyptans
- Asia (502)
    - indians
    - japanese
    - chinese
- Europe (390)
    - finns
    - english
    - italians

2. Выбрать базы (Поскольку образцы повторяются от базы к базе) содержащую макс кол-во образцов - потом максимум от того что осталось итд.
3. Перевести в бинарник, если необходимо.
4. Выровнять на последнюю версию генома.
5. Слить в один бинарный файл. 
6. Убрать лишние SNP.
7. Проверить пересечение SNP, отбирать там где больше 300 000. 
8. Установить семью как популяцию.
9. Сделать PCA на 10 первых компонент. Картинка с цветами и семьями.
10. Прогнать admixture. Сделать картинку на популяции в admixture

In [ ]:
!wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20200219.zip
!unzip plink_linux_x86_64_20200219.zip -d plink/

In [ ]:
#!pip install pandas
#!pip install tqdm

In [ ]:
!pip install pyliftover
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg19/liftOver/hg19ToHg38.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg18/liftOver/hg18ToHg38.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdbb/hg17/liftOver/hg17ToHg19.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg38/liftOver/hg38ToHg19.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg19/liftOver/hg19ToHg18.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg19/liftOver/hg19ToHg17.over.chain.gz
!wget http://hgdownload.cse.ucsc.edu/gbdb/hg18/liftOver/hg18ToHg38.over.chain.gz

In [1]:
import pandas as pd
import sys
#from os import listdir
#from os.path import isfile, join
#import codecs
import numpy as np
import os
import csv
from tqdm import tqdm
from pyliftover import LiftOver

In [2]:
df = pd.read_csv('Databases - individuals.csv')

In [3]:
df

,ID,population,Subpopulation,Region Macro,Region,Accuracy of pop,Project,Type of sequencing,directory,downloaded,Unnamed: 10,Unnamed: 11
0,Hazara6_27Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
1,Hazara6_2Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
2,Hazara6_66Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
3,Hazara6_68Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
4,Hazara6_6Af,hazaras,NaN,Asia,Asia Central,NaN,Estonian Biocenter,GENOTYPE,Estonian_Biocenter/afgan/,yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12223,SGVP00598,indians,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN
12224,SGVP00015,chinese,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN
12225,SGVP00016,chinese,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN
12226,SGVP00178,chinese,NaN,Asia,Asia Southeast,NaN,Singapore Genome Variation Project,NaN,SingaporeGVP/2009-0-30/,yes,NaN,NaN


In [4]:
listofpopulations = ['bantu', 'yoruba', 'egyptans', 'Egyptian', 'Indian', 'indians', 'chinese', 'japanese', 'finns', 'english', 'italians']

In [ ]:
id = [y for x in listofpopulations for y in df.ID[df.population == x]]
dr = [y for x in listofpopulations for y in df.directory[df.population == x]]
#print (id)
#print(dr)

In [28]:
len(id)

1282

**Распределение популяций по папкам**

In [6]:
drct = df[df.population.isin(listofpopulations)].groupby(by = 'directory').count().idxmax().ID
unique_dict = {}
best_count = 1
unique_dict = {x:y for x, y in zip(id, dr) if y == drct}
samples_in_directories =[[drct, len(unique_dict)]]
#unique_list(collect_sample)
cnt = 0
while  best_count > 0:
    dir_list = []
    for drct in df.directory[df.population.isin(listofpopulations)].unique():

        dir_list.append([drct, len([sample for sample in df.ID[df.population.isin(listofpopulations) & (df.directory == drct)] if sample not in unique_dict.keys()])])
        #print ('1 - ', drct, ' = ', len([sample for sample in df.ID[df.population.isin(listofpopulations) & (df.directory == drct)] if sample not in unique_dict.keys()]))
    best_count = max([int(x[1]) for x in dir_list])
    best_dir = [y[0] for y in dir_list if y[1] == best_count][0]
    if best_count ==0:
        break
    unique_dict.update({x:y for x, y in zip(id, dr) if (y == best_dir) & (x not in unique_dict)})
    samples_in_directories.append([best_dir, best_count])
    #print('6 - next step', best_dir, best_count)
    cnt +=1
    if cnt > 50:
        break

for record in samples_in_directories:
    print(record)

['ReichLab/AncietPresent2019/long', 558]
['SingaporeGVP/2009-0-30/', 179]
['ReichLab/AncietPresent2019/short', 152]
['SIBERIA/', 46]
['Estonian_Biocenter/Tamm_2019', 34]
['Estonian_Biocenter/khazar', 24]
['Jakobsson2008/', 20]
['Estonian_Biocenter/Tambets2018', 19]
['Estonian_Biocenter/jew/', 4]
['Estonian_Biocenter/Eichstaedt_2017', 3]
['Li2008/', 2]
['Estonian_Biocenter/america/', 1]


1. Готовим данные из папки Recihlab. Появилась новая 42 версия датасета (у нас 37). Обе версии выравненны на hg19 или 37 версию генома. Конвертация из формата проекта в бинарник происходит софтом eigensoft. 

# Dataset n.1

Устанавливаем eigensof

In [ ]:
!sudo apt-get install eigensoft

Закачиваем новую версию датасета

In [ ]:
!wget https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V42/V42.4/SHARE/public.dir/v42.4.1240K.tar

In [ ]:
!mkdir -p /home/cappelchi/ReichLab/V42/long/
!tar -xf v42.4.1240K.tar -C /home/cappelchi/ReichLab/V42/long

Готовим файл для конвертации

In [ ]:
!echo "genotypename:    /home/cappelchi/ReichLab/V42/long/v42.4.1240K.geno" > /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpname:        /home/cappelchi/ReichLab/V42/long/v42.4.1240K.snp" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivname:       /home/cappelchi/ReichLab/V42/long/v42.4.1240K.ind" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "outputformat:    PACKEDPED" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "genotypeoutname: /home/cappelchi/ReichLab/V42/long/v42.4.1240K.bed" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpoutname:      /home/cappelchi/ReichLab/V42/long/v42.4.1240K.bim" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivoutname:    /home/cappelchi/ReichLab/V42/long/v42.4.1240K.fam" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "familynames:     NO" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!cat /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED

In [ ]:
!convertf -p /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED

.fam files specification:
* Family ID ('FID')
* Within-family ID ('IID'; cannot be '0')
* Within-family ID of father ('0' if father isn't in dataset)
* Within-family ID of mother ('0' if mother isn't in dataset)
* Sex code ('1' = male, '2' = female, '0' = unknown)
* Phenotype value ('1' = control, '2' = case, '-9'/'0'/non-numeric = missing data if case/control)

In [8]:
df_fam = pd.read_csv(
    '/home/cappelchi/ReichLab/V42/long/v42.4.1240K.fam', 
    sep = '\s+', 
    names = ['ID1', 'ID2', 'IDF', 'IDM', 'sex', 'pop'], 
    header = None, 
    index_col = False
    )

Собираем названия образцов для выборки

In [9]:
Reich_long = [key for key in unique_dict if unique_dict[key] == 'ReichLab/AncietPresent2019/long']

Оказалось, что в новой базе названия образцов немного изменились. Корректируем названия для вырезания из датасета.

In [11]:
corr_names = [name for name in df_fam.ID2 if (name in Reich_long) or (name.split('.')[0] in Reich_long)]
print(len(corr_names))

531


Количество образцов немного уменьшилось. Смотрим какие образцы не вошли.

In [12]:
split_point = [name.split('.')[0] for name in df_fam.ID2 if (name in Reich_long) or (name.split('.')[0] in Reich_long)]
print(len(split_point))
rest = [name for name in Reich_long if name not in split_point]
print (rest)

531
['HGDP01028', 'HGDP01030', 'HGDP01034', 'HGDP01035', 'HGDP01414', 'HGDP01417', 'HGDP00928', 'HGDP00932', 'HGDP00936', 'IL-01', 'IL-04', 'IL-07', 'IL-08', 'IL-09', 'IL-10', 'IL-11', 'IL-12', 'IL-15', 'IL-16', 'HGDP00749', 'HGDP00773', 'HGDP01076', 'HGDP01078', 'HGDP01079', 'HGDP01163', 'HGDP01168', 'HGDP01172']


Апдейтим датасет под нашу выборку

In [ ]:
df_fam_selected = df_fam[df_fam.ID2.isin(corr_names)]

Создаём файл списка образцов под plink

In [ ]:
df_fam_selected.to_csv('/home/cappelchi/ReichLab/V42/long/keep.fam',
                      sep = ' ',
                       header = False,
                       index = False                       
                      )

Оставляем только нужные образцы и конвертируем в vcf

In [ ]:
!plink/plink --bfile /home/cappelchi/ReichLab/V42/long/v42.4.1240K --keep-fam /home/cappelchi/ReichLab/V42/long/keep.fam --make-bed --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected

In [ ]:
!plink/plink --bfile /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected  --keep-allele-order --recode vcf --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected

In [7]:
lo = LiftOver('hg19ToHg38.over.chain.gz')
lo38_19 = LiftOver('hg38ToHg19.over.chain.gz')
lo19_18 = LiftOver('hg19ToHg18.over.chain.gz')
lo19_17 = LiftOver('hg19ToHg17.over.chain.gz')

In [146]:
def recoord_vcf(input_name = '', output_name = ''):
    if (input_name == '') or (output_name == ''):
        return
    with open(input_name, 'r') as input_file:
        reader = csv.reader(input_file, delimiter = '\t')
        trigger = False
        cnt = 0
        cnt_crd = 0
        cnt_mis = 0
        with open(output_name, 'w') as output_file:
            for row in tqdm(reader):
                writer = csv.writer(output_file, delimiter = '\t')
                if trigger:
                    cnt += 1
                    rw = lo.convert_coordinate('chr' + str(row[0]), int(row[1]))
                    if rw:                   
                        row[1] = rw[0][1]
                        writer.writerow(row)
                        cnt_crd += 1
                    else:
                        cnt_mis +=1

                if row[0] == '#CHROM':
                    writer.writerow(row)
                    trigger = True

    print ('Cнипов перемещено = ', cnt_crd)
    print ('Снипов не найдено = ', cnt_mis)

In [ ]:
!plink/plink --vcf /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38.vcf --double-id --make-bed --out /home/cappelchi/ReichLab/V42/long/v42.4.1240K.selected_v38

# Dataset n.2

In [ ]:
!wget https://reichdata.hms.harvard.edu/pub/datasets/amh_repo/curated_releases/V42/V42.4/SHARE/public.dir/v42.4.1240K_HO.tar
!mkdir -p /home/cappelchi/ReichLab/V42/long/
!tar -xf v42.4.1240K_HO.tar -C /home/cappelchi/ReichLab/V42/long

In [ ]:
!echo "genotypename:    /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.geno" > /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpname:        /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.snp" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivname:       /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.ind" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "outputformat:    PACKEDPED" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "genotypeoutname: /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.bed" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "snpoutname:      /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.bim" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "indivoutname:    /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.fam" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!echo "familynames:     NO" >> /home/cappelchi/ReichLab/V42/long/par.EIGENSTRAT.PEDPACKED
!cat /home/cappelchi/ReichLab/V42/short/par.EIGENSTRAT.PEDPACKED

In [ ]:
!convertf -p /home/cappelchi/ReichLab/V42/short/par.EIGENSTRAT.PEDPACKED

In [129]:
df_fam = pd.read_csv(
    '/home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.fam', 
    sep = '\s+', 
    names = ['ID1', 'ID2', 'IDF', 'IDM', 'sex', 'pop'], 
    header = None, 
    index_col = False
    )

In [130]:
Reich_short = [key for key in unique_dict if unique_dict[key] == 'ReichLab/AncietPresent2019/short']

In [131]:
len(Reich_short)

152

In [135]:
df_fam_selected = df_fam[df_fam.ID2.isin(Reich_short)]

In [138]:
df_fam_selected.to_csv('/home/cappelchi/ReichLab/V42/short/keep.fam',
                      sep = ' ',
                       header = False,
                       index = False                       
                      )

In [ ]:
!plink/plink --bfile /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO --keep-fam /home/cappelchi/ReichLab/V42/short/keep.fam --make-bed --out /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected

In [ ]:
!plink/plink --bfile /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected  --keep-allele-order --recode vcf --out /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected

In [147]:
recoord_vcf(
    input_name = '/home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected.vcf', 
    output_name = '/home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected_38.vcf'
)

597603it [00:14, 41405.22it/s]

Cнипов перемещено =  593100
Снипов не найдено =  4473


In [ ]:
!plink/plink --vcf /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected_38.vcf --double-id --make-bed --out /home/cappelchi/ReichLab/V42/short/v42.4.1240K_HO.selected_38

# Dataset n.3

In [17]:
path = '/home/cappelchi/SingaporeGVP/QC/'
genotype1 = 'genotypes_chr' 
genotype2 = '_INS_15apr08_QC_b36_fwd'
genotype_ins= '_INS_15apr08_QC_b36_fwd'
genotype_chs = '_CHS_15apr08_QC_b36_fwd'
genotype_mas = '_MAS_15apr08_QC_b36_fwd'

In [8]:
def multi_snp(df_txt):
    #возвращает list с мульти SNP для запроса в онлайн базу
    to_del = []
    for cnt, allel in enumerate(df_txt.SNPalleles):    
        if len(allel) > 3:
            print (cnt, ' - ',df_txt.rs[cnt], ' - ', allel)
            to_del.append(df_txt.rs[cnt])
    return to_del

In [9]:
def del_multi_snp(to_del, input_file, output_file):
    #Удаляем мультиаллели
    with open(input_file, "r") as input:
        with open(output_file, "w") as output:
            for line in input:
                to_not_write = any(snp in to_del for snp in line.split('\t'))
                if not to_not_write:
                    #print (line.split('\t'))
                    output.write(line)        

In [ ]:
#for chrom in range(1, 23):    
#    df_txt = pd.read_csv(path + genotype1 + str(chrom) + genotype2 + '.txt', 
#                         #names = ['CHR', 'SNP', ' gen_dist', ' pos', 'ref', 'ALT'], 
#                         sep = '\s+', 
#                         header = 0, 
#                         index_col = False)
#    df_txt = df_txt.rename(columns={'rs#': 'rs'})
#    
#    multi_snps = multi_snp(df_txt)
#    
#    del_multi_snp(
#    multi_snps,
#    input_file  = path + genotype1 + str(chrom) + genotype2 + '.txt',
#    output_file = path + genotype1 + str(chrom) + genotype2 + '_cnv.txt'
#    )

In [188]:
#with open('/home/cappelchi/SingaporeGVP/QC/genotypes_chr1_INS_15apr08_QC_b36_fwd.txt', 'r') as input_file:
#    cnt = 0
#    reader = csv.reader(input_file, delimiter = '\t')
#    for line in reader:
#        print(line[0:12])
#        cnt += 1
#        if cnt > 5:
#            break

['rs#', 'SNPalleles', 'chrom', 'pos', 'strand', 'genome_build', 'center', 'protLSID', 'assayLSID', 'panelLSID', 'QC_code', 'SGVP00417']
['rs10458597', 'C/T', 'chr1', '554484', '+', 'ncbi_36', 'gis', 'AffymetrixSNP6', 'commercial.assay', 'Indian-83-unrelated', 'QC+', 'CC']
['rs9629043', 'C/T', 'chr1', '554636', '+', 'ncbi_36', 'gis', 'AffymetrixSNP6', 'commercial.assay', 'Indian-83-unrelated', 'QC+', 'CC']
['rs12565286', 'C/G', 'chr1', '711153', '+', 'ncbi_36', 'gis', 'AffymetrixSNP6', 'commercial.assay', 'Indian-83-unrelated', 'QC+', 'GG']
['rs12082473', 'A/G', 'chr1', '730720', '+', 'ncbi_36', 'gis', 'AffymetrixSNP6', 'commercial.assay', 'Indian-83-unrelated', 'QC+', 'GG']
['rs3094315', 'A/G', 'chr1', '742429', '+', 'ncbi_36', 'gis', 'Illumina1M_single', 'commercial.assay', 'Indian-83-unrelated', 'QC+', 'AG']


In [24]:
lo18_38 = LiftOver('hg18ToHg38.over.chain.gz')

In [10]:
def snp_recode(sample, ref, alt):
    #возвращает пару для VCF файла вида 0|0, на вход 'AA'
    if len(sample) == 2:
        snp_recode = str(2 * int(alt == sample[0]) - 1 + \
                         (ref == sample[0])).replace('-1', '.')  + '|' + \
                        str(2 * int(alt == sample[1]) - 1 + \
                            (ref == sample[1])).replace('-1', '.')
    else:
        snp_recode = '.|.'  
    
    return snp_recode

In [11]:
def recoord_vcf(input_name = '', output_name = ''):
    #Конвертирует txt файл в vcf требуется функция snp_recode
    if (input_name == '') or (output_name == ''):
        print ('File name missing')
        return
    header_vcf = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT']
    with open(input_name, 'r') as input_file:
        reader = csv.reader(input_file, delimiter = '\t')
        cnt_crd = 0
        cnt_mis = 0
        with open(output_name, 'w') as output_file:
            for cnt, row in tqdm(enumerate(reader)):
                #if cnt > 10:
                #    break
                writer = csv.writer(output_file, delimiter = '\t')
                
                if cnt == 0:
                    for sample in row[11:]:
                        header_vcf.append(sample)
                    writer.writerow(header_vcf)
                
                else:
                    #print(row[3], row[4])
                    pos = lo18_38.convert_coordinate(row[2], int(row[3]))
                    if (pos != []) & (len(row[1]) == 3):
                        ref = row[1][0]
                        alt = row[1][-1]
                        out_row = [row[2][3:], pos[0][1], row[0], ref, alt,  '.', 'PASS', '.', 'GT']
                        for smpl in row[11:]:
                            out_row.append(snp_recode(smpl, ref, alt))
                            
                        cnt_crd += 1
                        writer.writerow(out_row)
                    else:
                        cnt_mis += 1

    print ('Строк включено = ', cnt_crd)
    print ('Снипов не включено = ', cnt_mis)

### Индусы

1. Распаковываем

In [29]:
pop_dir_ins = 'INS/'
!mkdir {path}{pop_dir_ins}

In [ ]:
for chrom in range(1, 23):
    !gunzip -d -c {path}{genotype1}{chrom}{genotype_ins}.txt.gz > {path}{pop_dir_ins}{genotype1}{chrom}{genotype_ins}.txt

2. Переводим в VCF и обновляем координаты в 38 геном

In [267]:
for chrom in range(1, 23):
    recoord_vcf(
        input_name = path + genotype1 + str(chrom) + genotype2 + '.txt',
        output_name = path + genotype1 + str(chrom) + genotype2 + '_cnv_v38.vcf'
    )

115486it [00:14, 8230.15it/s]
849it [00:00, 8484.57it/s]

Строк включено =  115460
Снипов не включено =  25


117114it [00:14, 8172.75it/s]
801it [00:00, 8003.77it/s]

Строк включено =  117102
Снипов не включено =  11


97188it [00:12, 7919.12it/s]
840it [00:00, 8398.51it/s]

...

Строк включено =  36219
Снипов не включено =  1


19978it [00:02, 7520.60it/s]
829it [00:00, 8281.08it/s]

Строк включено =  19973
Снипов не включено =  4


20423it [00:03, 6367.92it/s]

Строк включено =  20404
Снипов не включено =  18


3. Конвертируем VCF to BED

In [ ]:
for chrom in range(1, 23):
    !plink/plink --vcf /home/cappelchi/SingaporeGVP/QC/genotypes_chr{chrom}_INS_15apr08_QC_b36_fwd_cnv_v38.vcf --make-bed --out /home/cappelchi/SingaporeGVP/QC/BED_INS/genotypes_INS_chr{chrom}

4. Создаем список и сливем всё в один бинарник 

In [ ]:
!echo /home/cappelchi/SingaporeGVP/QC/BED_INS/genotypes_INS_chr1 > /home/cappelchi/SingaporeGVP/QC/BED_INS/merge_ins.txt
for chrom in range(2, 23):
    !echo /home/cappelchi/SingaporeGVP/QC/BED_INS/genotypes_INS_chr{chrom} >> /home/cappelchi/SingaporeGVP/QC/BED_INS/merge_ins.txt
!cat /home/cappelchi/SingaporeGVP/QC/BED_INS/merge_ins.txt

In [ ]:
#Пробуем объединить все файлы в один набор
!plink/plink --threads 6 --bfile /home/cappelchi/SingaporeGVP/QC/BED_INS/genotypes_INS_chr1 --merge-list /home/cappelchi/SingaporeGVP/QC/BED_INS/merge_ins.txt --make-bed --out /home/cappelchi/SingaporeGVP/QC/BED_INS/genotypes_INS_merged

### Малазийцы

1. Распаковываем

In [31]:
pop_dir_mas = 'MAS/'
!mkdir {path}{pop_dir_mas}

In [38]:
for chrom in range(1, 23):
    !gunzip -d {path}{genotype1}{chrom}{genotype_mas}.txt.gz > {path}{pop_dir_mas}{genotype1}{chrom}{genotype_mas}.txt
    #!rm {path}{genotype1}{chrom}{genotype_chs}.txt > {path}{pop_dir_chs}{genotype1}{chrom}{genotype_chs}.txt

2. Переводим в VCF и обновляем координаты в 38 геном

In [41]:
for chrom in range(1, 23):
    recoord_vcf(
        input_name = path + genotype1 + str(chrom) + genotype_mas + '.txt',
        output_name = path + pop_dir_mas + genotype1 + str(chrom) + genotype_mas + '_cnv_v38.vcf'
    )

115056it [00:12, 9478.22it/s]
947it [00:00, 9457.29it/s]

Строк включено =  115030
Снипов не включено =  25


116930it [00:12, 9512.26it/s]
946it [00:00, 9452.64it/s]

Строк включено =  116917
Снипов не включено =  12


97017it [00:10, 9276.08it/s]
934it [00:00, 9334.31it/s]

...

Строк включено =  36096
Снипов не включено =  2


19948it [00:02, 9134.10it/s]
828it [00:00, 8271.98it/s]

Строк включено =  19943
Снипов не включено =  4


20458it [00:02, 8544.26it/s]

Строк включено =  20441
Снипов не включено =  16


3. Конвертируем VCF to BED

In [ ]:
for chrom in range(1, 23):
    !plink/plink --vcf {path}{pop_dir_mas}{genotype1}{chrom}{genotype_mas}_cnv_v38.vcf --make-bed --out {path}{pop_dir_mas}genotypes_MAS_chr{chrom}

4. Создаем список и сливем всё в один бинарник 

In [43]:
!echo {path}{pop_dir_mas}genotypes_MAS_chr1 > {path}{pop_dir_mas}merge_mas.txt
for chrom in range(2, 23):
    !echo {path}{pop_dir_mas}genotypes_MAS_chr{chrom} >> {path}{pop_dir_mas}merge_mas.txt
!cat {path}{pop_dir_mas}merge_mas.txt

/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr1
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr2
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr3
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr4
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr5
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr6
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr7
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr8
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr9
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr10
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr11
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr12
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr13
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr14
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr15
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr16
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr17
/home/cappelchi/SingaporeGVP/QC/MAS/genotypes_MAS_chr18
/

In [ ]:
#Пробуем объединить все файлы в один набор
!plink/plink --threads 6 --bfile {path}{pop_dir_mas}genotypes_MAS_chr1 --merge-list {path}{pop_dir_mas}merge_mas.txt --make-bed --out {path}{pop_dir_mas}genotypes_MAS_merged

### Китайцы

1. Распаковываем

In [13]:
pop_dir_chs = 'CHS/'
!mkdir {path}{pop_dir_chs}

In [21]:
for chrom in range(1, 23):
    !gunzip -d -c {path}{genotype1}{chrom}{genotype_chs}.txt.gz > {path}{pop_dir_chs}{genotype1}{chrom}{genotype_chs}.txt

2. Переводим в VCF и обновляем координаты в 38 геном

In [25]:
for chrom in range(1, 23):
    recoord_vcf(
        input_name = path + pop_dir_chs + genotype1 + str(chrom) + genotype_chs + '.txt',
        output_name = path + pop_dir_chs + genotype1 + str(chrom) + genotype_chs + '_cnv_v38.vcf'
    )

115410it [00:12, 8930.35it/s]
896it [00:00, 8956.81it/s]

Строк включено =  115383
Снипов не включено =  26


117093it [00:13, 8925.19it/s]
888it [00:00, 8869.75it/s]

Строк включено =  117080
Снипов не включено =  12


97243it [00:10, 8910.82it/s]
886it [00:00, 8853.63it/s]

...

Строк включено =  36231
Снипов не включено =  2


19970it [00:02, 8774.51it/s]
888it [00:00, 8872.25it/s]

Строк включено =  19965
Снипов не включено =  4


20400it [00:02, 8872.09it/s]

Строк включено =  20382
Снипов не включено =  17


3. Конвертируем VCF to BED

In [ ]:
for chrom in range(1, 23):
    !plink/plink --vcf {path}{pop_dir_chs}{genotype1}{chrom}{genotype_chs}_cnv_v38.vcf --make-bed --out {path}{pop_dir_chs}genotypes_CHS_chr{chrom}

4. Создаем список и сливем всё в один бинарник 

In [ ]:
!echo {path}{pop_dir_chs}genotypes_CHS_chr1 > {path}{pop_dir_chs}merge_chs.txt
for chrom in range(2, 23):
    !echo {path}{pop_dir_chs}genotypes_CHS_chr{chrom} >> {path}{pop_dir_chs}merge_chs.txt
!cat {path}{pop_dir_chs}merge_chs.txt

In [ ]:
#Пробуем объединить все файлы в один набор
!plink/plink --threads 6 --bfile {path}{pop_dir_chs}genotypes_CHS_chr1 --merge-list {path}{pop_dir_chs}merge_chs.txt --make-bed --out {path}{pop_dir_chs}genotypes_CHS_merged

- Сливаем три популяции в одну

In [30]:
!cp -f {path}{pop_dir_ins}genotypes_INS_merged*.* {path}{pop_dir_chs}

In [32]:
!cp -f {path}{pop_dir_mas}genotypes_MAS_merged*.* {path}{pop_dir_chs}

In [33]:
!echo {path}{pop_dir_chs}genotypes_CHS_merged > {path}{pop_dir_chs}all_merged.txt
!echo {path}{pop_dir_chs}genotypes_INS_merged >> {path}{pop_dir_chs}all_merged.txt
!echo {path}{pop_dir_chs}genotypes_MAS_merged >> {path}{pop_dir_chs}all_merged.txt
!cat {path}{pop_dir_chs}all_merged.txt

/home/cappelchi/SingaporeGVP/QC/CHS/genotypes_CHS_merged
/home/cappelchi/SingaporeGVP/QC/CHS/genotypes_INS_merged
/home/cappelchi/SingaporeGVP/QC/CHS/genotypes_MAS_merged


In [ ]:
#Пробуем объединить все файлы в один набор
!plink/plink --threads 6 --bfile {path}{pop_dir_chs}genotypes_CHS_merged --merge-list {path}{pop_dir_chs}all_merged.txt --make-bed --out {path}{pop_dir_chs}ALL_genotypes_merged

- Оставляем только требуемые образцы

In [35]:
df_fam = pd.read_csv(
    path + pop_dir_chs + 'ALL_genotypes_merged.fam', 
    sep = '\s+', 
    names = ['ID1', 'ID2', 'IDF', 'IDM', 'sex', 'pop'], 
    header = None, 
    index_col = False
    )

In [36]:
df_fam

,ID1,ID2,IDF,IDM,sex,pop
0,SGVP00002,SGVP00002,0,0,0,-9
1,SGVP00010,SGVP00010,0,0,0,-9
2,SGVP00011,SGVP00011,0,0,0,-9
3,SGVP00014,SGVP00014,0,0,0,-9
4,SGVP00015,SGVP00015,0,0,0,-9
...,...,...,...,...,...,...
263,SGVP00582,SGVP00582,0,0,0,-9
264,SGVP00589,SGVP00589,0,0,0,-9
265,SGVP00592,SGVP00592,0,0,0,-9
266,SGVP00596,SGVP00596,0,0,0,-9


In [37]:
SingaporeGVP = [key for key in unique_dict if unique_dict[key] == 'SingaporeGVP/2009-0-30/']

In [39]:
len(SingaporeGVP)

179

In [40]:
df_fam_selected = df_fam[df_fam.ID1.isin(SingaporeGVP)]

In [43]:
df_fam_selected.to_csv (path + pop_dir_chs + 'all_keep.fam',
                      sep = ' ',
                       header = False,
                       index = False                       
                      )

In [ ]:
!plink/plink --bfile {path}{pop_dir_chs}ALL_genotypes_merged --keep-fam {path}{pop_dir_chs}all_keep.fam --make-bed --out {path}{pop_dir_chs}SingaporeGVP_merged_selected_updated

## Dataset n.4